# Anomalies zonales et transitoires - transports méridiens

**Auteur : FERRY Frédéric (DESR/ENM/C3M) - septembre 2022

Les fichiers de données au format netcdf (données quotidiennes ERA5 du géopotentiel, de la température, du vent zonal et du vent méridien à 500hPa pour l'année 2021) doivent être téléchargés et placés dans le répertoire data :

- https://cds.climate.copernicus.eu/cdsapp#!/dataset/reanalysis-era5-pressure-levels?tab=form

Il faut télécharger des données tri-horaires puis calculer les moyennes quotidiennes via la commande CDO daymean :

- https://code.mpimet.mpg.de/projects/cdo/embedded/index.html#x1-930002.2.6

--> fichiers era5_z500_daily_2021.nc, era5_t500_daily_2021.nc, era5_u500_daily_2021.nc, era5_v500_daily_2021.nc

In [ ]:
%matplotlib inline

import os
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt

from cartopy.util import add_cyclic_point
import cartopy.crs as ccrs
from cartopy.mpl.geoaxes import GeoAxes
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter

from netCDF4 import Dataset
import h5py

In [ ]:
def lonflip(da):
    lon_name = 'longitude'
    da['_longitude_adjusted'] = xr.where(da[lon_name] > 180, da[ lon_name] - 360, da[lon_name])
    da = (
        da .swap_dims({lon_name: '_longitude_adjusted'})
        .sel(**{'_longitude_adjusted': sorted(da._longitude_adjusted)})
        .drop(lon_name))
    da = da.rename({'_longitude_adjusted': lon_name})
    return da

# Traitement des données

In [ ]:
diri="./daily_era5/"

fz    = xr.open_dataset(diri+"era5_z500_daily_2021.nc")
lat  = fz.latitude.values
print(fz)

z = fz['z']/9.81
z=lonflip(z)

lon  = z.longitude.values

print(z.shape)

<div class="alert alert-warning">
    
Moyenne temporelle : $$ \overline{z} = \frac{1}{T} \int_0^T z dt $$
    
</div>

In [ ]:
date1='2021-01-01'
date2='2021-03-31'

z_season=z.sel(time=slice(date1,date2))
print(z_season)

zbar = z_season.mean('time')
print(zbar.shape)

<div class="alert alert-warning">

Moyenne zonale : $$ \left[ z \right] = \frac{1}{2\pi} \int_0^{2\pi} z d\lambda $$

Anomalie temporelle : $$ z^\prime = z - \overline{z} $$

Anomalie zonale : $$ z^* = z - \left[ z \right] $$
    
</div>

In [ ]:
zzmean=z_season.mean('longitude')
z_season, zzmean, = xr.broadcast(z_season,zzmean)
zstar=z_season-zzmean
zprime = z_season - zbar

print(zprime.shape)
print(zzmean.shape)
print(zstar.shape)

In [ ]:
date='2021-01-01'
zday = z_season.sel(time=date).squeeze()
zprimeday = zprime.sel(time=date).squeeze()
zzmeanday = zzmean.sel(time=date).squeeze()
zstarday = zstar.sel(time=date).squeeze()
print(zday)

In [ ]:
proj=ccrs.EqualEarth()

def plot_background(ax):
    ax.coastlines()
    ax.gridlines(crs=ccrs.PlateCarree(), linewidth=0.5, color='gray', alpha=0.5, linestyle='-')
    return ax

In [ ]:
levels_z = np.arange(4800,6050,50)
levels_zanom = np.arange(-500,550,50)
title='Geopotential height at 500 hPa : '+date


fig = plt.figure(figsize=(19,7))
fig.suptitle(title, fontsize=16)

ax=fig.add_subplot(231, projection=proj)
plot_background(ax)
ax.set_title("z", fontsize=14)
cf = ax.contourf(lon, lat, zday, levels_z, transform=ccrs.PlateCarree(), cmap='jet', extend='both')
c = ax.contour(lon, lat, zday, levels_z, colors= 'black', linewidths=0.2, transform=ccrs.PlateCarree())
cb = fig.colorbar(cf, orientation='horizontal', aspect=65, shrink=1, pad=0.20, extendrect='True')
cb.set_label('m', size='small')

ax=fig.add_subplot(232, projection=proj)
plot_background(ax)
ax.set_title("$[z]$", fontsize=14)
cf = ax.contourf(lon, lat, zzmeanday, levels_z, transform=ccrs.PlateCarree(), cmap='jet', extend='both')
c = ax.contour(lon, lat, zzmeanday, levels_z, colors= 'black', linewidths=0.2, transform=ccrs.PlateCarree())
cb = fig.colorbar(cf, orientation='horizontal', aspect=65, shrink=1, pad=0.20, extendrect='True')
cb.set_label('m', size='small')

ax=fig.add_subplot(233, projection=proj)
plot_background(ax)
ax.set_title("$z^* = z-[z]$", fontsize=14)
cf = ax.contourf(lon, lat, zstarday, levels_zanom, transform=ccrs.PlateCarree(), cmap='coolwarm', extend='both')
c = ax.contour(lon, lat, zstarday, levels_zanom, colors= 'black', linewidths=0.2, transform=ccrs.PlateCarree())
cb = fig.colorbar(cf, orientation='horizontal', aspect=65, shrink=1, pad=0.20, extendrect='True')
cb.set_label('m', size='small')

ax=fig.add_subplot(234, projection=proj)
plot_background(ax)
ax.set_title("z", fontsize=14)
cf = ax.contourf(lon, lat, zday, levels_z, transform=ccrs.PlateCarree(), cmap='jet', extend='both')
c = ax.contour(lon, lat, zday, levels_z, colors= 'black', linewidths=0.2, transform=ccrs.PlateCarree())
cb = fig.colorbar(cf, orientation='horizontal', aspect=65, shrink=1, pad=0.20, extendrect='True')
cb.set_label('m', size='small')


ax=fig.add_subplot(235, projection=proj)
plot_background(ax)
ax.set_title("$\overline{z}$", fontsize=14)
cf = ax.contourf(lon, lat, zbar, levels_z, transform=ccrs.PlateCarree(), cmap='jet', extend='both')
c = ax.contour(lon, lat, zbar, levels_z, colors= 'black', linewidths=0.2, transform=ccrs.PlateCarree())
cb = fig.colorbar(cf, orientation='horizontal', aspect=65, shrink=1, pad=0.20, extendrect='True')
cb.set_label('m', size='small')


ax=fig.add_subplot(236, projection=proj)
plot_background(ax)
ax.set_title("$z' = z-\overline{z}$", fontsize=14)
cf = ax.contourf(lon, lat, zprimeday, levels_zanom, transform=ccrs.PlateCarree(), cmap='coolwarm', extend='both')
c = ax.contour(lon, lat, zprimeday, levels_zanom, colors= 'black', linewidths=0.2, transform=ccrs.PlateCarree())
cb = fig.colorbar(cf, orientation='horizontal', aspect=65, shrink=1, pad=0.20, extendrect='True')
cb.set_label('m', size='small')


figname='./figs/z'+date
fig.savefig(figname+'.png',bbox_inches='tight')

In [ ]:
fig = plt.figure(figsize=(15,10))
fig.suptitle(title, fontsize=16)

ax=fig.add_subplot(111, projection=proj)
plot_background(ax)
ax.set_title("z", fontsize=14)
cf = ax.contourf(lon, lat, zday, levels_z, transform=ccrs.PlateCarree(), cmap='jet', extend='both')
c = ax.contour(lon, lat, zday, levels_z, colors= 'black', linewidths=0.2, transform=ccrs.PlateCarree())
cb = fig.colorbar(cf, orientation='horizontal', aspect=65, shrink=1, pad=0.20, extendrect='True')
cb.set_label('m', size='small')

figname='./figs/z0_'+date
fig.savefig(figname+'.png',bbox_inches='tight')

In [ ]:
fig = plt.figure(figsize=(15,10))
fig.suptitle(title, fontsize=16)

ax=fig.add_subplot(111, projection=proj)
plot_background(ax)
ax.set_title("$[z]$", fontsize=14)
cf = ax.contourf(lon, lat, zzmeanday, levels_z, transform=ccrs.PlateCarree(), cmap='jet', extend='both')
c = ax.contour(lon, lat, zzmeanday, levels_z, colors= 'black', linewidths=0.2, transform=ccrs.PlateCarree())
cb = fig.colorbar(cf, orientation='horizontal', aspect=65, shrink=1, pad=0.20, extendrect='True')
cb.set_label('m', size='small')

figname='./figs/z1_'+date
fig.savefig(figname+'.png',bbox_inches='tight')

In [ ]:
fig = plt.figure(figsize=(15,10))
fig.suptitle(title, fontsize=16)

ax=fig.add_subplot(111, projection=proj)
plot_background(ax)
ax.set_title("$z^* = z-[z]$", fontsize=14)
cf = ax.contourf(lon, lat, zstarday, levels_zanom, transform=ccrs.PlateCarree(), cmap='coolwarm', extend='both')
c = ax.contour(lon, lat, zstarday, levels_zanom, colors= 'black', linewidths=0.2, transform=ccrs.PlateCarree())
cb = fig.colorbar(cf, orientation='horizontal', aspect=65, shrink=1, pad=0.20, extendrect='True')
cb.set_label('m', size='small')

figname='./figs/z2_'+date
fig.savefig(figname+'.png',bbox_inches='tight')

In [ ]:
fig = plt.figure(figsize=(15,10))
fig.suptitle(title, fontsize=16)

ax=fig.add_subplot(111, projection=proj)
plot_background(ax)
ax.set_title("$\overline{z}$", fontsize=14)
cf = ax.contourf(lon, lat, zbar, levels_z, transform=ccrs.PlateCarree(), cmap='jet', extend='both')
c = ax.contour(lon, lat, zbar, levels_z, colors= 'black', linewidths=0.2, transform=ccrs.PlateCarree())
cb = fig.colorbar(cf, orientation='horizontal', aspect=65, shrink=1, pad=0.20, extendrect='True')
cb.set_label('m', size='small')

figname='./figs/z3_'+date
fig.savefig(figname+'.png',bbox_inches='tight')

In [ ]:
fig = plt.figure(figsize=(15,10))
fig.suptitle(title, fontsize=16)

ax=fig.add_subplot(111, projection=proj)
plot_background(ax)
ax.set_title("$z' = z-\overline{z}$", fontsize=14)
cf = ax.contourf(lon, lat, zprimeday, levels_zanom, transform=ccrs.PlateCarree(), cmap='coolwarm', extend='both')
c = ax.contour(lon, lat, zprimeday, levels_zanom, colors= 'black', linewidths=0.2, transform=ccrs.PlateCarree())
cb = fig.colorbar(cf, orientation='horizontal', aspect=65, shrink=1, pad=0.20, extendrect='True')
cb.set_label('m', size='small')

figname='./figs/z4_'+date
fig.savefig(figname+'.png',bbox_inches='tight')

<div class="alert alert-danger">

    
<b>Coder la décomposition suivante : </b>
    
$$z=\overline{[z]}+\overline{z^*}+[z]'+z^{*'}$$

Composante axisymétrique et stationnaire de la circulation : $$\overline{[z]}$$
Ecart stationnaire à la moyenne zonale : $$\overline{z^*}$$
Perturbation transitoire de la moyenne zonale : $$[z]'$$
Perturbation transitoire non axisymétrique : $$z^{*'}$$

</div>

In [ ]:
zzmeanbar =
zstarbar =                      
zzmeanprime =
zstarprime =

print(zstarbar.shape)
print(zzmeanbar.shape)
print(zzmeanprime.shape)
print(zstarprime.shape)

In [ ]:
date='2021-01-01'
zday = z_season.sel(time=date).squeeze()
zzmeanprimeday = zzmeanprime.sel(time=date).squeeze()
zstarprimeday = zstarprime.sel(time=date).squeeze()
sum=zzmeanbar+zstarbar+zzmeanprimeday+zstarprimeday
print(zzmeanprimeday.shape)

In [ ]:
levels_z = np.arange(4800,6050,50)
levels_zanom = np.arange(-300,320,20)

fig = plt.figure(figsize=(19,7))
fig.suptitle(title, fontsize=16)

ax=fig.add_subplot(231, projection=proj)
plot_background(ax)
ax.set_title("z", fontsize=14)
cf = ax.contourf(lon, lat, zday, levels_z, transform=ccrs.PlateCarree(), cmap='jet', extend='both')
c = ax.contour(lon, lat, zday, levels_z, colors= 'black', linewidths=0.2, transform=ccrs.PlateCarree())
cb = fig.colorbar(cf, orientation='horizontal', aspect=65, shrink=1, pad=0.20, extendrect='True')
cb.set_label('m', size='small')

ax=fig.add_subplot(232, projection=proj)
plot_background(ax)
ax.set_title("$\overline{[z]}$", fontsize=14)
cf = ax.contourf(lon, lat, zzmeanbar, levels_z, transform=ccrs.PlateCarree(), cmap='jet', extend='both')
c = ax.contour(lon, lat, zzmeanbar, levels_z, colors= 'black', linewidths=0.2, transform=ccrs.PlateCarree())
cb = fig.colorbar(cf, orientation='horizontal', aspect=65, shrink=1, pad=0.20, extendrect='True')
cb.set_label('m', size='small')

ax=fig.add_subplot(233, projection=proj)
plot_background(ax)
ax.set_title("$\overline{z^*}$", fontsize=14)
cf = ax.contourf(lon, lat, zstarbar, levels_zanom, transform=ccrs.PlateCarree(), cmap='coolwarm', extend='both')
c = ax.contour(lon, lat, zstarbar, levels_zanom, colors= 'black', linewidths=0.2, transform=ccrs.PlateCarree())
cb = fig.colorbar(cf, orientation='horizontal', aspect=65, shrink=1, pad=0.20, extendrect='True')
cb.set_label('m', size='small')

ax=fig.add_subplot(234, projection=proj)
plot_background(ax)
ax.set_title("$[z]'$", fontsize=14)
cf = ax.contourf(lon, lat, zzmeanprimeday, levels_zanom, transform=ccrs.PlateCarree(), cmap='coolwarm', extend='both')
c = ax.contour(lon, lat, zzmeanprimeday, levels_zanom, colors= 'black', linewidths=0.2, transform=ccrs.PlateCarree())
cb = fig.colorbar(cf, orientation='horizontal', aspect=65, shrink=1, pad=0.20, extendrect='True')
cb.set_label('m', size='small')


ax=fig.add_subplot(235, projection=proj)
plot_background(ax)
ax.set_title("$z^{*'}$", fontsize=14)
cf = ax.contourf(lon, lat, zstarprimeday, levels_zanom, transform=ccrs.PlateCarree(), cmap='coolwarm', extend='both')
c = ax.contour(lon, lat, zstarprimeday, levels_zanom, colors= 'black', linewidths=0.2, transform=ccrs.PlateCarree())
cb = fig.colorbar(cf, orientation='horizontal', aspect=65, shrink=1, pad=0.20, extendrect='True')
cb.set_label('m', size='small')


ax=fig.add_subplot(236, projection=proj)
plot_background(ax)
ax.set_title("$\overline{[z]}+\overline{z^*}+[z]'+z^{*'}$", fontsize=14)
cf = ax.contourf(lon, lat, sum, levels_z, transform=ccrs.PlateCarree(), cmap='jet', extend='both')
c = ax.contour(lon, lat, sum, levels_z, colors= 'black', linewidths=0.2, transform=ccrs.PlateCarree())
cb = fig.colorbar(cf, orientation='horizontal', aspect=65, shrink=1, pad=0.20, extendrect='True')
cb.set_label('m', size='small')


figname='./figs/z_decomp_'+date
fig.savefig(figname+'.png',bbox_inches='tight')

In [ ]:
fig = plt.figure(figsize=(15,10))
fig.suptitle(title, fontsize=16)

ax=fig.add_subplot(111, projection=proj)
plot_background(ax)
ax.set_title("z", fontsize=14)
cf = ax.contourf(lon, lat, zday, levels_z, transform=ccrs.PlateCarree(), cmap='jet', extend='both')
c = ax.contour(lon, lat, zday, levels_z, colors= 'black', linewidths=0.2, transform=ccrs.PlateCarree())
cb = fig.colorbar(cf, orientation='horizontal', aspect=65, shrink=1, pad=0.20, extendrect='True')
cb.set_label('m', size='small')

figname='./figs/z_decomp0_'+date
fig.savefig(figname+'.png',bbox_inches='tight')

In [ ]:
fig = plt.figure(figsize=(15,10))
fig.suptitle(title, fontsize=16)

ax=fig.add_subplot(111, projection=proj)
plot_background(ax)
ax.set_title("$\overline{[z]}$", fontsize=14)
cf = ax.contourf(lon, lat, zzmeanbar, levels_z, transform=ccrs.PlateCarree(), cmap='jet', extend='both')
c = ax.contour(lon, lat, zzmeanbar, levels_z, colors= 'black', linewidths=0.2, transform=ccrs.PlateCarree())
cb = fig.colorbar(cf, orientation='horizontal', aspect=65, shrink=1, pad=0.20, extendrect='True')
cb.set_label('m', size='small')

figname='./figs/z_decomp1_'+date
fig.savefig(figname+'.png',bbox_inches='tight')

In [ ]:
fig = plt.figure(figsize=(15,10))
fig.suptitle(title, fontsize=16)

ax=fig.add_subplot(111, projection=proj)
plot_background(ax)
ax.set_title("$\overline{z^*}$", fontsize=14)
cf = ax.contourf(lon, lat, zstarbar, levels_zanom, transform=ccrs.PlateCarree(), cmap='coolwarm', extend='both')
c = ax.contour(lon, lat, zstarbar, levels_zanom, colors= 'black', linewidths=0.2, transform=ccrs.PlateCarree())
cb = fig.colorbar(cf, orientation='horizontal', aspect=65, shrink=1, pad=0.20, extendrect='True')
cb.set_label('m', size='small')

figname='./figs/z_decomp2_'+date
fig.savefig(figname+'.png',bbox_inches='tight')

In [ ]:
fig = plt.figure(figsize=(15,10))
fig.suptitle(title, fontsize=16)

ax=fig.add_subplot(111, projection=proj)
plot_background(ax)
ax.set_title("$[z]'$", fontsize=14)
cf = ax.contourf(lon, lat, zzmeanprimeday, levels_zanom, transform=ccrs.PlateCarree(), cmap='coolwarm', extend='both')
c = ax.contour(lon, lat, zzmeanprimeday, levels_zanom, colors= 'black', linewidths=0.2, transform=ccrs.PlateCarree())
cb = fig.colorbar(cf, orientation='horizontal', aspect=65, shrink=1, pad=0.20, extendrect='True')
cb.set_label('m', size='small')

figname='./figs/z_decomp3_'+date
fig.savefig(figname+'.png',bbox_inches='tight')

In [ ]:
fig = plt.figure(figsize=(15,10))
fig.suptitle(title, fontsize=16)

ax=fig.add_subplot(111, projection=proj)
plot_background(ax)
ax.set_title("$z^{*'}$", fontsize=14)
cf = ax.contourf(lon, lat, zstarprimeday, levels_zanom, transform=ccrs.PlateCarree(), cmap='coolwarm', extend='both')
c = ax.contour(lon, lat, zstarprimeday, levels_zanom, colors= 'black', linewidths=0.2, transform=ccrs.PlateCarree())
cb = fig.colorbar(cf, orientation='horizontal', aspect=65, shrink=1, pad=0.20, extendrect='True')
cb.set_label('m', size='small')

figname='./figs/z_decomp4_'+date
fig.savefig(figname+'.png',bbox_inches='tight')

# Transports méridiens de chaleur

In [ ]:
ft    = xr.open_dataset(diri+"era5_t500_daily_2021.nc")
fv    = xr.open_dataset(diri+"era5_v500_daily_2021.nc")
fu    = xr.open_dataset(diri+"era5_u500_daily_2021.nc")

lat  = fv.latitude.values
print(ft)

t = ft['t']
v = fv['v']
u = fu['u']

t=lonflip(t)
v=lonflip(v)
u=lonflip(u)

lon  = t.longitude.values

print(t.shape)

In [ ]:
date1='2021-01-01'
date2='2021-03-31'

In [ ]:
u_season = u.sel(time=slice(date1,date2))
v_season = v.sel(time=slice(date1,date2))
T0_season = t.sel(time=slice(date1,date2))
T_season_global = T0_season.mean(dim='time').weighted(np.cos(np.deg2rad(T0_season.latitude))).mean(dim=('latitude','longitude'))
T_season=T0_season-T_season_global
print(v_season.shape)
print(T_season.shape)

<div class="alert alert-danger">
  
<b>Calculer les flux méridiens de chaleur : </b>
    
<br>

Transport méridiens de chaleur total :
$$[\overline{vT}]$$    
    
<br>
    
Transport méridien de chaleur par la circulation en moyenne zonale :
$$\overline{[v]} \overline{[T]}$$

<br>

Transport méridien de chaleur par les perturbations transitoires :
$$[\overline{v'T'}]$$

<br>
    
Transport méridien de chaleur  par les circulations stationnaires non axisymétriques :
$$[\overline{v^*}\overline{T^*}]$$

        
</div>

In [ ]:
vTbar = 
vTbarzmean = 

vbar = 
Tbar = 
vbarzmean = 
Tbarzmean = 
vbarzmean_Tbarzmean = 

vprime = 
Tprime = 
vprimeTprimebar = 
vprimeTprimebarzmean = 

vbarstar = 
Tbarstar = 
vbarstarTbarstarzmean = 

<div class="alert alert-danger">
  
<b>Calculer les flux méridiens de quantité de mouvement zonal : </b>
    
<br>

Transport méridiens de qdm total :
$$[\overline{uv}]$$    
    
<br>
    
Transport méridien de qdm par la circulation en moyenne zonale :
$$\overline{[u]} \overline{[v]}$$

<br>

Transport méridien de qdm par les perturbations transitoires :
$$[\overline{u'v'}]$$

<br>
    
Transport méridien de qdm  par les circulations stationnaires non axisymétriques :
$$[\overline{u^*}\overline{v^*}]$$

        
</div>

In [ ]:
uvbar = 
uvbarzmean = 

vbar = 
ubar = 
vbarzmean = 
ubarzmean = 
ubarzmean_vbarzmean = 

vprime = 
uprime = 
uprimevprimebar = 
uprimevprimebarzmean = 

vbarstar = 
ubarstar = 
ubarstarvbarstarzmean = 

In [ ]:
scale=np.cos(lat*np.pi/180.)

total_vt = vTbarzmean
mean_meridional_vt = vbarzmean_Tbarzmean
transient_eddy_vt = vprimeTprimebarzmean
stationary_eddy_vt = vbarstarTbarstarzmean

print(total_vt.shape)

In [ ]:
total_uv = uvbarzmean
mean_meridional_uv = ubarzmean_vbarzmean
transient_eddy_uv = uprimevprimebarzmean
stationary_eddy_uv = ubarstarvbarstarzmean

print(total_uv.shape)

In [ ]:
fig = plt.figure(figsize=(15,10))

plt.xlabel('Latitude')
plt.ylabel('Transport (K m/s)')
plt.xticks(np.arange(-90, 120, 30), ('90S', '60S', '30S', 'Eq', '30N', '60N', '90N'))
plt.title('Mean meridional heat transport decomposition at 500 hPa: '+date1+'-'+date2)
plt.axhline(0, color='grey', linestyle="--")
plt.axvline(0, color='grey', linestyle="--")
plt.xlim(-90,90)
plt.plot(lat, total_vt, color='k', linewidth=3, label='Total : $[\overline{vT}]$')
plt.plot(lat, mean_meridional_vt, color='red', label='Mean meridional : $[\overline{v}][\overline{T}]$')
plt.plot(lat, transient_eddy_vt, color='blue', label='Transient eddy : $[\overline{v^\prime T^\prime}]$')
plt.plot(lat, stationary_eddy_vt, color='green', label='Stationary eddy : $[\overline{v}^* \overline{T}^*]$')
plt.legend(loc='best')

figname='./figs/eddy_heat_flux_decomposition_'+date1+'-'+date2
plt.savefig(figname+'.png',bbox_inches='tight')
plt.show()

In [ ]:
fig = plt.figure(figsize=(15,10))

plt.xlabel('Latitude')
plt.ylabel('Transport ($m^2/s^2$)')
plt.xticks(np.arange(-90, 120, 30), ('90S', '60S', '30S', 'Eq', '30N', '60N', '90N'))
plt.title('Mean meridional zonal momentum transport decomposition at 500 hPa: '+date1+'-'+date2)
plt.axhline(0, color='grey', linestyle="--")
plt.axvline(0, color='grey', linestyle="--")
plt.xlim(-90,90)
plt.plot(lat, total_uv, color='k', linewidth=3, label='Total : $[\overline{uv}]$')
plt.plot(lat, mean_meridional_uv, color='red', label='Mean meridional : $[\overline{u}][\overline{v}]$')
plt.plot(lat, transient_eddy_uv, color='blue', label='Transient eddy : $[\overline{u^\prime v^\prime}]$')
plt.plot(lat, stationary_eddy_uv, color='green', label='Stationary eddy : $[\overline{u}^* \overline{v}^*]$')
plt.legend(loc='best')

figname='./figs/eddy_momentum_flux_decomposition_'+date1+'-'+date2
plt.savefig(figname+'.png',bbox_inches='tight')
plt.show()

<div class="alert alert-danger">
  
<b>Tracer les mêmes courbes pour une saison d'été</b>
        
</div>